In [1]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [5]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix_three'

/content/drive/My Drive/Colab Notebooks/dw_matrix_three


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
      )
  return model

In [0]:
def predict(trained_model, X_test, y_test, scoring=accuracy_score):
  y_pred_probe = trained_model.predict(X_test)
  y_pred = np.argmax(y_pred_probe, axis=1)

  y_test_norm = np.argmax(y_test, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
                    
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(num_classes, activation='softmax')
  ])

In [13]:
model = get_cnn_v5(input_shape, num_classes)
trained_model = train_model(model, X_train, y_train)
predict(trained_model, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.3808 - accuracy: 0.3347 - val_loss: 0.8365 - val_accuracy: 0.7425
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.7521 - accuracy: 0.7633 - val_loss: 0.2170 - val_accuracy: 0.9316
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3795 - accuracy: 0.8828 - val_loss: 0.0862 - val_accuracy: 0.9766
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2654 - accuracy: 0.9197 - val_loss: 0.0566 - val_accuracy: 0.9835
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2124 - accuracy: 0.9382 - val_loss: 0.0452 - val_accuracy: 0.9869


0.9668934240362812

In [14]:
trained_model.evaluate(X_test, y_test)[0]

138/138 [==============================] - 0s 3ms/step - loss: 0.1291 - accuracy: 0.9669


[0.12909173965454102, 0.9668934345245361]

In [0]:
def get_model(params):
  return Sequential([
                    
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),

      Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_one']),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_two']),

      Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
      )
  
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [30]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.6),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.6),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.6),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30, 
    Trials()
)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.5590104695116228, 'dropout_cnn_block_three': 0.4759904651601974, 'dropout_cnn_block_two': 0.5430309803932474, 'dropout_dense_block_one': 0.604193371904622, 'dropout_dense_block_two': 0.5819416036704415}
accuracy=0.4204081594944
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.5192441105529065, 'dropout_cnn_block_three': 0.3265104084975424, 'dropout_cnn_block_two': 0.38275081688297097, 'dropout_dense_block_one': 0.5137026203858879, 'dropout_dense_block_two': 0.6589712882675703}
accuracy=0.8863945603370667
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4164511247056037, 'dropout_cnn_block_three': 0.5080986912926945, 'dropout_cnn_block_two': 0.5882656243138641, 'dropout_dense_block_one': 0.38598824034502577, 'dropout_dense_block_two': 0.6911976801622537}
accuracy=0.6818594336509705
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.3082737611032839, 'dropout_cnn_block_three': 0.5586296107575326, 'dropout_cnn_block_two': 0.33372708981026866,